In [ ]:
!nvidia-smi

### Installing packages
**Reastart the kernel after installing packages.**

In [ ]:
# ITNOA
%%capture
try:
  import hazm
  print('Alll packages are installed!')
except:
  !pip install -U datasets
  #!pip install -U transformers
  !pip install transformers[torch]
  !pip install evaluate
  !pip install torchaudio
  !pip install librosa
  !pip install jiwer
  !pip install hazm
  !pip install num2fawords
  !pip install wandb

In [ ]:
#!apt install git-lfs

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
model_name_or_path = "masoudmzb/wav2vec2-xlsr-multilingual-53-fa"  # base model to fine-tune on
save_dir = "/content/drive/MyDrive/wav2vec2-large-xlsr-persian-asr-shemo"

In [ ]:
from transformers.trainer_utils import get_last_checkpoint
import os
last_checkpoint = None
if os.path.exists(save_dir):
    last_checkpoint = get_last_checkpoint(save_dir)
print(last_checkpoint if last_checkpoint else str(None))

None


In [ ]:
# Login with your wandb api
%env WANDB_PROJECT = Felfel
!wandb login <PASTE_YOUR_API_HERE> --relogin

env: WANDB_PROJECT=Felfel
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from datasets import load_dataset, load_metric

import pandas as pd
import numpy as np

import hazm
from num2fawords import words, ordinal_words
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import os
import string
import six
import re
import glob

In [ ]:
_normalizer = hazm.Normalizer()

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬",'ٔ', ",", "?",
    ".", "!", "-", ";", ":",'"',"“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š',
     #"ء",
]

# In case of farsi
chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",

    # "ها": "  ها", "ئ": "ی",

    "a": " ای ", "b": " بی ", "c": " سی ", "d": " دی ", "e": " ایی ", "f": " اف ",
    "g": " جی ", "h": " اچ ", "i": " آی ", "j": " جی ", "k": " کی ", "l": " ال ",
    "m": " ام ", "n": " ان ", "o": " او ", "p": " پی ", "q": " کیو ", "r": " آر ",
    "s": " اس ", "t": " تی ", "u": " یو ", "v": " وی ", "w": " دبلیو ", "x": " اکس ",
    "y": " وای ", "z": " زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))

def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text

def text_normalizer(text, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = text.lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)

    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None

    return text + " "

def batch_normalizer(batch, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = batch["sentence"].lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)

    text = " ".join(_text) + " "
    text = text.strip()

    batch["sentence"] = text + " "
    return batch

In [ ]:
#@title Kaggle Credential

import json

username = "" #@param {type: "string"}
api_key = "" #@param {type: "string"}


if username and api_key:
    token = {"username": username, "key": api_key}

    !mkdir ~/.kaggle
    !mkdir /content/.kaggle
    with open('/content/.kaggle/kaggle.json', 'w') as f:
        json.dump(token, f)

    !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 /root/.kaggle/kaggle.json

    print('Your are ready to use kaggle API!')


In [ ]:
%%capture
import os
if not os.path.exists('/content/stuff/shemo'):
  !mkdir -p /content/stuff/shemo
  !kaggle datasets download mansourehk/shemo-persian-speech-emotion-detection-database
  !unzip shemo-persian-speech-emotion-detection-database.zip -d /content/stuff/shemo/
  !wget -O stuff/shemo/shemo.json https://github.com/pariajm/sharif-emotional-speech-dataset/raw/master/shemo.json

In [ ]:
abs_path_to_data = os.path.join("/content/stuff", "shemo")
!ls {abs_path_to_data}

female	female.zip  male  male.zip  shemo.json


In [ ]:
import json
shemo_json_file = open(abs_path_to_data + '/shemo.json')
shemo_json = json.load(shemo_json_file)
shemo_json_file.close()

In [ ]:
data = []
for _id, val in tqdm(shemo_json.items()):
    male_path = f"{abs_path_to_data}/male/{_id}.wav"
    female_path = f"{abs_path_to_data}/female/{_id}.wav"
    text = val['transcript']
    if "M" in _id and os.path.exists(male_path):
        data.append({
            "_id": _id,
            "sentence": text,
            "path": male_path.strip(),
            "gender": "M"
        })
    elif "F" in _id and os.path.exists(female_path):
        data.append({
            "_id": _id,
            "sentence": text,
            "path": female_path.strip(),
            "gender": "F"
        })

100%|██████████| 3000/3000 [00:00<00:00, 202822.61it/s]


In [ ]:
from IPython.display import display, HTML

df = pd.DataFrame(data)
df_samples = pd.DataFrame(data[:10])
display(HTML(df_samples.to_html()))

,_id,sentence,path,gender
0,M36S02,﻿عشق هر دوی ما رو به مرز بی‌قراری و جنون کشونده بود,/content/stuff/shemo/male/M36S02.wav,M
1,F21N23,جلوی پیرهن اینجا است. این پاپیون مال پشت پیرهنته.,/content/stuff/shemo/female/F21N23.wav,F
2,M21F01,اون مال جوونیامه,/content/stuff/shemo/male/M21F01.wav,M
3,M28A43,و مقام و منسب دوکی را از خانوادۀ ما گرفتن,/content/stuff/shemo/male/M28A43.wav,M
4,F26N02,می‌دونین که این هفته نوبت کشیک شب شوهرمه.,/content/stuff/shemo/female/F26N02.wav,F
5,F24H15,اون هم ببینه اینقدر این کار رو دوست داری باهات همکاری می‌کنه.,/content/stuff/shemo/female/F24H15.wav,F
6,F24H01,بیمار دچار نوعی بیماری قلبی پیشرفته در قسمت دریچه‎های قلب است.,/content/stuff/shemo/female/F24H01.wav,F
7,M02N14,زندگی ایشون در این مدت بیم بود و امید,/content/stuff/shemo/male/M02N14.wav,M
8,F24A54,اصلاً تو کی هستی؟,/content/stuff/shemo/female/F24A54.wav,F
9,M28H02,با این حال خیال می‏کنم نتیجۀ نقشۀ تو این می‎شه که فردا من رو با خواری و خفت از اینجا بیرون کنن,/content/stuff/shemo/male/M28H02.wav,M


In [ ]:
df["sentence"] = df["sentence"].apply(lambda t: text_normalizer(t))  # Normalizing!
df.head()

,_id,sentence,path,gender
0,M36S02,عشق هر دوی ما رو به مرز بی قراری و جنون کشونده...,/content/stuff/shemo/male/M36S02.wav,M
1,F21N23,جلوی پیرهن اینجا است این پاپیون مال پشت پیرهنته,/content/stuff/shemo/female/F21N23.wav,F
2,M21F01,اون مال جوونیامه,/content/stuff/shemo/male/M21F01.wav,M
3,M28A43,و مقام و منسب دوکی را از خانواده ما گرفتن,/content/stuff/shemo/male/M28A43.wav,M
4,F26N02,می دونین که این هفته نوبت کشیک شب شوهرمه,/content/stuff/shemo/female/F26N02.wav,F


In [ ]:
'''import torchaudio
import librosa
import IPython.display as IPythonDisplay
import numpy as np

idx = np.random.randint(0, len(df))
print(idx)
sample = df.iloc[idx]

path = sample["path"]
print(sample["sentence"], "\n")
speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()

speech = librosa.resample(np.asarray(speech), orig_sr = sr, target_sr = 16_000)
IPythonDisplay.Audio(data=np.asarray(speech), autoplay=False, rate=16000)'''

'import torchaudio\nimport librosa\nimport IPython.display as IPythonDisplay\nimport numpy as np\n\nidx = np.random.randint(0, len(df))\nprint(idx)\nsample = df.iloc[idx]\n\npath = sample["path"]\nprint(sample["sentence"], "\n")\nspeech, sr = torchaudio.load(path)\nspeech = speech[0].numpy().squeeze()\n\nspeech = librosa.resample(np.asarray(speech), orig_sr = sr, target_sr = 16_000)\nIPythonDisplay.Audio(data=np.asarray(speech), autoplay=False, rate=16000)'

In [ ]:
#random_state is set to 313, so trin/test split is always the same as before.
train_df, test_df = train_test_split(df, test_size = 0.1, random_state = 313, stratify = df["gender"])

train_df = train_df[["path", "sentence"]]
train_df = train_df.reset_index(drop=True)

test_df = test_df[["path", "sentence"]]
test_df = test_df.reset_index(drop=True)

print(train_df.shape)
print(test_df.shape)
print(train_df['sentence'][0])
print(test_df['sentence'][0])

(2554, 2)
(284, 2)
چون کویی کوئک روزه بود و فکر می کردم که روزه اش تا غروب آفتاب طول می کشه تصمیم گرفتم که تا غروب در شهر گردش کنم 
از طرفی شنیدیم که امیررضا شریفی جانباز ورزشکار دارای کارت اهدای عضو هنگام خروج از باشگاه دچار سانحه می شه و جان می بازه 


In [ ]:
print(abs_path_to_data)
train_df.to_csv(f"{abs_path_to_data}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{abs_path_to_data}/test.csv", sep="\t", encoding="utf-8", index=False)

/content/stuff/shemo


In [ ]:
shemo_train = load_dataset("csv", data_files={"train": abs_path_to_data + "/train.csv"}, delimiter="\t")["train"]
shemo_test = load_dataset("csv", data_files={"test": abs_path_to_data + "/test.csv"}, delimiter="\t")["test"]

print(shemo_train)
print(shemo_test)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'sentence'],
    num_rows: 2554
})
Dataset({
    features: ['path', 'sentence'],
    num_rows: 284
})


In [ ]:
def speech_file_to_array_fn(batch):
    import torchaudio, librosa, numpy as np  # WHY did I add this?????????? @lnxdx
    target_sampling_rate = 16_000            # And this?
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=target_sampling_rate)

    batch["speech"] = speech_array
    batch["sampling_rate"] = target_sampling_rate
    batch["duration_in_seconds"] = len(batch["speech"]) / target_sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [ ]:
shemo_train = shemo_train.map(speech_file_to_array_fn,  remove_columns=shemo_train.column_names, num_proc=4)#, batched=True)
shemo_test = shemo_test.map(speech_file_to_array_fn,  remove_columns=shemo_test.column_names, num_proc=4)#, batched=True)

Map (num_proc=4):   0%|          | 0/2554 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/284 [00:00<?, ? examples/s]

In [ ]:
shemo_train

Dataset({
    features: ['speech', 'sampling_rate', 'duration_in_seconds', 'target_text'],
    num_rows: 2554
})

In [ ]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)

preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

In [ ]:
text = "از مهمونداری کنار بکشم"
print(" ".join(processor.tokenizer.tokenize(text)))
print(processor.decode(processor.tokenizer.encode(text)))

ا ز | م ه م و ن د ا ر ی | ک ن ا ر | ب ک ش م
از مهمونداری کنار بکشم


In [ ]:
def prepare_dataset(batch, processor):
    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"]).input_values
    batch["labels"] = processor(text = batch["target_text"]).input_ids
    return batch

In [ ]:
shemo_train = shemo_train.map(prepare_dataset, fn_kwargs={"processor": processor}, remove_columns=shemo_train.column_names, num_proc=4)#, batch_size=8, batched=True)
shemo_test = shemo_test.map(prepare_dataset, fn_kwargs={"processor": processor}, remove_columns=shemo_test.column_names, num_proc=4)#, batch_size=8, batched=True)

Map (num_proc=4):   0%|          | 0/2554 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/284 [00:00<?, ? examples/s]

In [ ]:
shemo_train

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 2554
})

In [ ]:
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods.
        input_features = [{"input_values": feature["input_values"][0]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        labels_batch = self.processor.pad(labels=label_features, padding=self.padding, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding='longest')

In [ ]:
import evaluate
wer_metric = evaluate.load("wer")

In [ ]:
import random

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    '''if isinstance(label_str, list):
        if isinstance(pred_str, list) and len(pred_str) == len(label_str):
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str[index]}"')

        else:
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str}"')'''

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC
'''
# hp-shemo: Hyperparams of 'm3hrdadfi/wav2vec2-large-xlsr-persian-shemo' (Original version of this notebook which belongs to Mehrdad Farahani)
attention_dropout = 0.1,
hidden_dropout = 0.1,
feat_proj_dropout = 0.0,
mask_time_prob = 0.05,
layerdrop = 0.1,
ctc_loss_reduction = 'mean',
ctc_zero_infinity = True,

# hp0: Hyperparams of 'facebook/wav2vec2-large-xlsr-53' or 'facebook/wav2vec2-xls-r-300m'
attention_dropout = 0.1,
hidden_dropout = 0.1,
feat_proj_dropout = 0.1,
mask_time_prob = 0.075,
layerdrop = 0.1,
ctc_loss_reduction = 'sum',
ctc_zero_infinity = False,

# hp-base: Hyperparams simmilar to hp0 ('facebook/wav2vec2-large-xlsr-53' or 'facebook/wav2vec2-xls-r-300m')
attention_dropout = 0.1,
hidden_dropout = 0.1,
feat_proj_dropout = 0.1,
mask_time_prob = 0.075,
layerdrop = 0.1,
ctc_loss_reduction = 'mean',
ctc_zero_infinity = True,

# hp-mehrdad: Hyperparams of 'm3hrdadfi/wav2vec2-large-xlsr-persian-v3'
attention_dropout = 0.05316,  # ~0.05
hidden_dropout    = 0.01941,  # ~0.02
feat_proj_dropout = 0.01249,  # ~0.01
mask_time_prob    = 0.04529,  # ~0.045
layerdrop         = 0.01377,  # ~0.01
ctc_loss_reduction = 'mean',
ctc_zero_infinity = True,

# hp-masoud: Hyperparams of 'masoudmzb/wav2vec2-xlsr-multilingual-53-fa'
attention_dropout = 0.2,
hidden_dropout = 0.2,
feat_proj_dropout = 0.1,
mask_time_prob = 0.2,
layerdrop = 0.2,
ctc_loss_reduction = 'mean',
ctc_zero_infinity = True,

# hp1: Hyperparams of https://huggingface.co/blog/fine-tune-xlsr-wav2vec2 (XLS-R model, not XLSR)
attention_dropout = 0.0,
hidden_dropout = 0.0,
feat_proj_dropout = 0.0,
mask_time_prob = 0.05,
layerdrop = 0.0,
ctc_loss_reduction="mean",
ctc_zero_infinity = True,  # Except this one! This parameter in the above link is 'False'.

# hp-myself-2:
attention_dropout = 0.08,   # 0.1   in hp-base    ~0.05  in hp-mehrdad
hidden_dropout    = 0.06,   # 0.1   in hp-base    ~0.02  in hp-mehrdad
feat_proj_dropout = 0.05,   # 0.1   in hp-base    ~0.01  in hp-mehrdad
mask_time_prob    = 0.05,   # 0.075 in hp-base    ~0.045 in hp-mehrdad
layerdrop         = 0.05,   # 0.1   in hp-base    ~0.01  in hp-mehrdad
ctc_loss_reduction = 'mean',
ctc_zero_infinity = True,
'''
model = Wav2Vec2ForCTC.from_pretrained(
    model_name_or_path if not last_checkpoint else last_checkpoint,

    # hp-mehrdad: Hyperparams of 'm3hrdadfi/wav2vec2-large-xlsr-persian-v3'
    attention_dropout = 0.05316,  # ~0.05
    hidden_dropout    = 0.01941,  # ~0.02
    feat_proj_dropout = 0.01249,  # ~0.01
    mask_time_prob    = 0.04529,  # ~0.045
    layerdrop         = 0.01377,  # ~0.01
    ctc_loss_reduction = 'mean',
    ctc_zero_infinity = True,

    bos_token_id=processor.tokenizer.bos_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer.get_vocab()) # 40
)

config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore.
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_encoder()

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

Now, all instances can be passed to Trainer and we are ready to start training!



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take between 180 and 240 minutes depending on the GPU allocated to this notebook. While the trained model yields somewhat satisfying results on *Common Voice*'s test data of Turkish, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how XLSR-Wav2Vec2's [checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) can be fine-tuned on a low-resource ASR dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [ ]:
#import torch
#torch.cuda.empty_cache()
#print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
print(f"last_checkpoint: {last_checkpoint}")

last_checkpoint: None


In [ ]:
from transformers import TrainingArguments

run_name         = "20_2000_1e-5_hp-mehrdad"  # Late changed to 'Wav2Vec2-Large-XLSR-Persian-ShEMO'.
num_steps        = 2000
alpha            = 1e-5
train_batch_size = 8


training_args = TrainingArguments(  # 2554 / 8 (batch size) = 319.25 batches   284
    output_dir=save_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate = alpha,
    warmup_steps = 500,
    #num_train_epochs = 13,
    max_steps = num_steps,  # was 2000
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=100,
    eval_steps=100,
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    run_name = run_name,
    hub_model_id = run_name,
    report_to = "wandb"
    #push_to_hub = True
)

#Now, all instances can be passed to Trainer and we are ready to start training!

from transformers import Trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=shemo_train,
    eval_dataset=shemo_test,
    tokenizer=processor,
)


if last_checkpoint:
    print(f"last_checkpoint: {last_checkpoint}")
    train_result = trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    train_result = trainer.train()

wandb: Currently logged in as: logxdx. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,0.752300,0.661245,0.314836
200,0.777300,0.662657,0.316881
300,0.778400,0.652472,0.318341
400,0.717200,0.659145,0.315421
500,0.719900,0.676473,0.312500
600,0.735200,0.647367,0.318633
700,0.710100,0.656922,0.315129
800,0.723000,0.658257,0.316589
900,0.708400,0.664186,0.315421
1000,0.701300,0.661848,0.316297


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [ ]:
metrics = train_result.metrics
max_train_samples = len(shemo_train)
metrics["train_samples"] = min(max_train_samples, len(shemo_train))

#trainer.save_model()

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =         6.25
  total_flos               = 4468823431GF
  train_loss               =       0.7323
  train_runtime            =   1:19:44.32
  train_samples            =         2554
  train_samples_per_second =        3.344
  train_steps_per_second   =        0.209


In [ ]:
#model.save_pretrained('123abc')
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

'https://huggingface.co/lnxdx/6-2_1000_1e-5_hp-myself-2/tree/main/'